<a href="https://colab.research.google.com/github/RonaldTuna/TwitterSAD/blob/main/Full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This data science project will be a fully fleshed out web application that will:
  

1.   Scrape data from the web
2.   preprocess the data
3.   Train a model 
4.   Model the data
5.   
6.   Have web accessable dashboard 

And use popular tools such as:

1. Tableau
2. SQL ("MySQL")
3. AWS
4. Flair
5. Flask
6. Python
7. Machine Learning


So far the plan is:
 to scrape twitter using python and twitter related libraries
 use SQL to create tables with the date, and sentiment analysis values
 Create a dashboard that displays information and visualizations
 host the application online so that users may specify hashtags and time frames.
 ...maybe try to link relevant popular posts?




DOWNLOAD RELEVANT LIBRARIES

In [ ]:
# install, set connection
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/
# query using %sql or %%sql
df = %sql SELECT Host, User, authentication_string FROM mysql.user
df

 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.
     |████████████████████████████████| 43 kB 2.4 MB/s 
 * mysql+pymysql://root:***@/


,Host,User,authentication_string
0,localhost,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B
1,localhost,mysql.session,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
2,localhost,mysql.sys,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
3,localhost,debian-sys-maint,*58FD287CD2CD16C12E7CA365AAF84F21C9FDB7CC


In [ ]:
import torch
!pip install flair
import flair

In [ ]:
pip install Tweepy

In [ ]:
pip show tweepy

Name: tweepy
Version: 3.10.0
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: requests-oauthlib, six, requests
Required-by: 


In [ ]:
#IMPORT LIBRARIES
import os
#import mysql.connector
#from mysql.connector import Error
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import tweepy
import torch
import flair
import csv
%matplotlib inline

In [ ]:
#Establish a connection to the "MySQL Community Server" - NOT USING AT THE MOMENT

def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print("Error: '{err}'")

    return connection

In [ ]:
#access database
conn = sqlite3.connect('tweet.db')
print("Opened database successfully");

#create and specify table fields
conn.execute('''
CREATE TABLE IF NOT EXISTS team_data(hashtag text, 
                      tweet text, 
                      year integer, 
                      month integer, 
                      day integer, 
                      hour integer, 
                      sentiment double);''')

conn.commit()

print("Table created successfully");

conn.close()

Opened database successfully
Table created successfully


In [ ]:
#scrape twitter for tweets and store them in the above table
# function to display data of each tweet
def printtweetdata(n, ith_tweet):
        print()
        print(f"Tweet {n}:")
        print(f"Username:{ith_tweet[0]}")
        print(f"Description:{ith_tweet[1]}")
        print(f"Location:{ith_tweet[2]}")
        print(f"Following Count:{ith_tweet[3]}")
        print(f"Follower Count:{ith_tweet[4]}")
        print(f"Total Tweets:{ith_tweet[5]}")
        print(f"Retweet Count:{ith_tweet[6]}")
        print(f"Tweet Text:{ith_tweet[7]}")
        print(f"Hashtags Used:{ith_tweet[8]}")
        print(f"Date created:{ith_tweet[9]}")
 
 
# function to perform data extraction
def scrape(words, date_since, numtweet):
 
        # Creating DataFrame using pandas
        db = pd.DataFrame(columns=['username',
                                   'description',
                                   'location',
                                   'following',
                                   'followers',
                                   'totaltweets',
                                   'retweetcount',
                                   'text',
                                   'hashtags',
                                   'date'])
 
        # We are using .Cursor() to search
        # through twitter for the required tweets.
        # The number of tweets can be
        # restricted using .items(number of tweets)
        tweets = tweepy.Cursor(api1.search,
                               words, lang="en",
                               since_id=date_since,
                               tweet_mode='extended').items(numtweet)
 
        # .Cursor() returns an iterable object. Each item in
        # the iterator has various attributes
        # that you can access to
        # get information about each tweet
        list_tweets = [tweet for tweet in tweets]
 
        # Counter to maintain Tweet Count
        i = 1
 
        # we will iterate over each tweet in the
        # list for extracting information about each tweet
        for tweet in list_tweets:
                username = tweet.user.screen_name
                description = tweet.user.description
                location = tweet.user.location
                following = tweet.user.friends_count
                followers = tweet.user.followers_count
                totaltweets = tweet.user.statuses_count
                retweetcount = tweet.retweet_count
                hashtags = tweet.entities['hashtags']
                date = tweet.created_at
 
                # Retweets can be distinguished by
                # a retweeted_status attribute,
                # in case it is an invalid reference,
                # except block will be executed
                try:
                        text = tweet.retweeted_status.full_text
                except AttributeError:
                        text = tweet.full_text
                hashtext = list()
                for j in range(0, len(hashtags)):
                        hashtext.append(hashtags[j]['text'])
 
                # Here we are appending all the
                # extracted information in the DataFrame
                ith_tweet = [username, description,
                             location, following,
                             followers, totaltweets,
                             retweetcount, text,
                             hashtext, date]
                db.loc[len(db)] = ith_tweet
 
                # Function call to print tweet data on screen
                printtweetdata(i, ith_tweet)
                i = i+1
        filename = 'scraped_tweets.csv'
 
        # we will save our database as a CSV file.
        db.to_csv(filename)
 
if __name__ == '__main__':
 
        # Enter your own credentials obtained
        # from your developer account
        consumer_key = "xAPNXy86WutJKZgrtnhiW6mDc"
        consumer_secret = "d1nyzfkVI2uGylGyYqraVum1RODriAmlvApHkJbNdWtAbYsgzo"
        access_key = "1511407274958352386-tH9wCzGN2ToD19EawiNOfwRPDcEcek"
        access_secret = "F7EcsOmfYimLCDpuUdUdFtkhnPImIIpM2aUjZGPLzszTK"
 
 
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_key, access_secret)
        api1 = tweepy.API(auth)
 
        # Enter Hashtag and initial date
        print("Enter Twitter HashTag to search for")
        words = input()
        print("Enter Date since The Tweets are required in yyyy-mm-dd")
        date_since = input()
 
        # number of tweets you want to extract in one run
        numtweet = 100
        scrape(words, date_since, numtweet)
        print('Scraping has completed!')

#run sentiment analysis on tweets

Enter Twitter HashTag to search for
Dingle
Enter Date since The Tweets are required in yyyy-mm-dd
2022-04-10

Tweet 1:
Username:TammyTu78097268
Description:https://t.co/DI0fam6vZZ
Location:Crown Heights, Brooklyn
Following Count:15144
Follower Count:18561
Total Tweets:471169
Retweet Count:0
Tweet Text:Sailors Do Care By Sinbad -Advanced Proof Copy 1936  A.E. Dingle
https://t.co/O7ePpXpY5u
Hashtags Used:[]
Date created:2022-04-11 23:40:22

Tweet 2:
Username:shihojansson
Description:a pixel artist
Location:ChariotShiho#4690
Following Count:1406
Follower Count:209
Total Tweets:5126
Retweet Count:0
Tweet Text:@WistfulHopes Spindle dingle and bingle
Hashtags Used:[]
Date created:2022-04-11 23:40:19

Tweet 3:
Username:coralina_anna
Description:
Location:
Following Count:474
Follower Count:309
Total Tweets:8205
Retweet Count:0
Tweet Text:@gabstevent @FredericoACR dingle
Hashtags Used:[]
Date created:2022-04-11 23:33:04

Tweet 4:
Username:amoc7777
Description:
Location:Dublin
Following Count:2

Here we will organize the data above and place them into more concise tables that contain information relevant(Tweet text, hashtags, date) to the graphs being displayed on the dashboard using SQL and a sentiment analysis library called FLAIR

In [ ]:
from sqlite3.dbapi2 import Date
data = pd.read_csv("scraped_tweets.csv")
df = pd.DataFrame(data)

conn = pymysql.connect(
        host='localhost',
        user='root', 
        password = '108137395'
        )
cursor = conn.cursor()

cursor.execute('''
		CREATE TABLE TwitterSA (
			date text,
			product_name nvarchar(50),
			tweet_text 
			)
               ''')
#CREATE TABLE TwitterSA (
#  Date DATE
#  Hashtags varchar(255)
#  Text varchar(255)
#);
#df.to_sql("TwitterSA", conn, if_exists='append', index=False)
#con = sqlite3.connect("sqlite:///TwitterSA.db")
#cur = con.cursor()

OperationalError: ignored

The below code creates the websites using a web framework called FLASK


In [1]:
git init

SyntaxError: ignored